# RAGEx for RAG

basierend auf: https://dl.acm.org/doi/pdf/10.1145/3626772.3657660

In [1]:
import sys, json
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [2]:
from src.modules.rag_ex_explainable import RAGExExplainable
from src.modules.cot_explainable import _format_documents
from src.modules.rag_engine import RAGEngine
from src.modules.llm_client import LLMClient
from src.modules.data_loader_single_hop import BoolQDataLoader

### Real data example

In [3]:
_dataLoader = BoolQDataLoader()
documents = _dataLoader.setup()

In [4]:
ragEngine = RAGEngine(persist_dir="../data/vector_db_bool")
ragEngine.setup(documents=documents)

Loading existing vector store from ../data/vector_db_bool...
RagEngine ready.


In [20]:
client = LLMClient(provider="ollama", model_name="gemma3:1b")

question = "Where does Sam live?"
#
#documents = ragEngine.retrieve_documents(question)
#documents = [documents[0]]
#
#print(documents)
#context = _format_documents(documents, from_query=question)
context = 'My Name is Sam. I live in Regensburg. I like bavarian cuisine.'
extra = """
Output rules (MANDATORY):
- Respond with exactly one sentence
- Respond with only the answer, nothing more
- No punctuation
- No explanations
- No line breaks
- No additional text

"""
message = client._create_final_answer_prompt(question, context, extra=extra)

llm = client.get_llm()
final_answer_response = llm.invoke(message)
final_answer = final_answer_response.content.strip()

print(final_answer)

explainer = RAGExExplainable(llm_client=client)
result = explainer.explain(query=question, answer=final_answer, context=context)

Connecting to local Ollama (gemma3:1b)...
Sam lives in Regensburg.
Pertubating 1 of 12
Pertubating 2 of 12
Pertubating 3 of 12
Pertubating 4 of 12
Pertubating 5 of 12
Pertubating 6 of 12
Pertubating 7 of 12
Pertubating 8 of 12
Pertubating 9 of 12
Pertubating 10 of 12
Pertubating 11 of 12
Pertubating 12 of 12


In [21]:
print(RAGExExplainable.prettify(result))

token	importance	strategies
My	0.000	leave_one_out:raw=0.000,sim=1.000 | random_noise:raw=0.000,sim=1.000
Name	0.000	leave_one_out:raw=0.000,sim=1.000 | random_noise:raw=0.000,sim=1.000
is	0.000	leave_one_out:raw=0.000,sim=1.000 | random_noise:raw=0.000,sim=1.000
Sam.	0.393	leave_one_out:raw=0.261,sim=0.739 | random_noise:raw=0.261,sim=0.739
I	0.000	leave_one_out:raw=0.000,sim=1.000 | random_noise:raw=0.000,sim=1.000
live	0.410	leave_one_out:raw=0.273,sim=0.727 | random_noise:raw=0.000,sim=1.000
in	0.674	leave_one_out:raw=0.448,sim=0.552 | random_noise:raw=0.000,sim=1.000
Regensburg.	1.000	leave_one_out:raw=0.665,sim=0.335 | random_noise:raw=0.623,sim=0.377
I	0.000	leave_one_out:raw=0.000,sim=1.000 | random_noise:raw=0.000,sim=1.000
like	0.220	leave_one_out:raw=0.146,sim=0.854 | random_noise:raw=0.000,sim=1.000
bavarian	0.000	leave_one_out:raw=0.000,sim=1.000 | random_noise:raw=0.000,sim=1.000
cuisine.	0.000	leave_one_out:raw=0.000,sim=1.000 | random_noise:raw=0.000,sim=1.000
